In [1]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import pickle
import re

In [2]:
movies_final_2018_df = pd.read_pickle('df_all_movies_2018_pk')
movies_final_2018_df.head(10)

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url
0,1,Black Panther,"Feb 16, 2018",Walt Disney,Action,"$700,059,566","76,845,177",https://www.the-numbers.com/movie/Black-Panthe...,https://www.the-numbers.com/movie/Black-Panthe...
1,2,Avengers: Infinity War,"Apr 27, 2018",Walt Disney,Action,"$678,815,482","74,513,225",https://www.the-numbers.com/movie/Avengers-Inf...,https://www.the-numbers.com/movie/Avengers-Inf...
2,3,Incredibles 2,"Jun 15, 2018",Walt Disney,Adventure,"$608,581,744","66,803,704",https://www.the-numbers.com/movie/Incredibles-...,https://www.the-numbers.com/movie/Incredibles-...
3,4,Jurassic World: Fallen Kingdom,"Jun 22, 2018",Universal,Action,"$417,719,760","45,852,883",https://www.the-numbers.com/movie/Jurassic-Wor...,https://www.the-numbers.com/movie/Jurassic-Wor...
4,5,Deadpool 2,"May 18, 2018",20th Century Fox,Action,"$324,512,774","35,621,600",https://www.the-numbers.com/movie/Deadpool-2#t...,https://www.the-numbers.com/movie/Deadpool-2#t...
5,6,Dr. Seussâ The Grinch,"Nov 9, 2018",Universal,Adventure,"$266,280,410","29,229,463",https://www.the-numbers.com/movie/Dr-Seuss-The...,https://www.the-numbers.com/movie/Dr-Seuss-The...
6,7,Jumanji: Welcome to the Jungle,"Dec 20, 2017",Sony Pictures,Adventure,"$235,506,359","25,851,411",https://www.the-numbers.com/movie/Jumanji-Welc...,https://www.the-numbers.com/movie/Jumanji-Welc...
7,8,Mission: ImpossibleâFallout,"Jul 27, 2018",Paramount Pictures,Action,"$220,159,104","24,166,751",https://www.the-numbers.com/movie/Mission-Impo...,https://www.the-numbers.com/movie/Mission-Impo...
8,9,Ant-Man and the Wasp,"Jul 6, 2018",Walt Disney,Action,"$216,648,740","23,781,420",https://www.the-numbers.com/movie/Ant-Man-and-...,https://www.the-numbers.com/movie/Ant-Man-and-...
9,10,Solo: A Star Wars Story,"May 25, 2018",Walt Disney,Adventure,"$213,767,512","23,465,149",https://www.the-numbers.com/movie/Solo-A-Star-...,https://www.the-numbers.com/movie/Solo-A-Star-...


In [3]:
legs_2018_df = pd.read_pickle('legs_2018_df_pk')
legs_2018_df['Legs']

0      3.47
1      2.63
2      3.33
3      2.82
4      2.59
       ... 
731    3.45
732    1.77
733    1.00
734    3.27
735    2.27
Name: Legs, Length: 736, dtype: float64

In [4]:
movies_final_2018_df['Legs'] = legs_2018_df['Legs']

In [5]:
movies_final_2018_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs
0,1,Black Panther,"Feb 16, 2018",Walt Disney,Action,"$700,059,566","76,845,177",https://www.the-numbers.com/movie/Black-Panthe...,https://www.the-numbers.com/movie/Black-Panthe...,3.47
1,2,Avengers: Infinity War,"Apr 27, 2018",Walt Disney,Action,"$678,815,482","74,513,225",https://www.the-numbers.com/movie/Avengers-Inf...,https://www.the-numbers.com/movie/Avengers-Inf...,2.63
2,3,Incredibles 2,"Jun 15, 2018",Walt Disney,Adventure,"$608,581,744","66,803,704",https://www.the-numbers.com/movie/Incredibles-...,https://www.the-numbers.com/movie/Incredibles-...,3.33
3,4,Jurassic World: Fallen Kingdom,"Jun 22, 2018",Universal,Action,"$417,719,760","45,852,883",https://www.the-numbers.com/movie/Jurassic-Wor...,https://www.the-numbers.com/movie/Jurassic-Wor...,2.82
4,5,Deadpool 2,"May 18, 2018",20th Century Fox,Action,"$324,512,774","35,621,600",https://www.the-numbers.com/movie/Deadpool-2#t...,https://www.the-numbers.com/movie/Deadpool-2#t...,2.59
...,...,...,...,...,...,...,...,...,...,...
731,732,To Die is Hard,"Dec 17, 2010",Driving With Our …,Comedy,$580,63,https://www.the-numbers.com/movie/To-Die-is-Ha...,https://www.the-numbers.com/movie/To-Die-is-Ha...,3.45
732,733,Higher Power,"May 11, 2018",Magnet Pictures,Thriller/Suspense,$528,57,https://www.the-numbers.com/movie/Higher-Power...,https://www.the-numbers.com/movie/Higher-Power...,1.77
733,734,Mobile Homes,"Apr 20, 2018",Mongrel Media,Drama,$449,49,https://www.the-numbers.com/movie/Mobile-Homes...,https://www.the-numbers.com/movie/Mobile-Homes...,1.00
734,735,Midget Zombie Takeover,"Feb 8, 2013",Driving With Our …,Comedy,$402,44,https://www.the-numbers.com/movie/Midget-Zombi...,https://www.the-numbers.com/movie/Midget-Zombi...,3.27


In [6]:
budget_2018_df = pd.read_pickle('budget_2018_pk')
wbo_to_budget_ratio_2018_df = pd.read_pickle('wbo_to_budget_ratio_2018_pk')

movies_final_2018_df['Budget'] = budget_2018_df['Production_Budget']
movies_final_2018_df['WBO_to_Budget_Ratio'] = (
    wbo_to_budget_ratio_2018_df['WBO to Budget Ratio'])

In [7]:
movies_final_2018_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs,Budget,WBO_to_Budget_Ratio
0,1,Black Panther,"Feb 16, 2018",Walt Disney,Action,"$700,059,566","76,845,177",https://www.the-numbers.com/movie/Black-Panthe...,https://www.the-numbers.com/movie/Black-Panthe...,3.47,200000000,6.7
1,2,Avengers: Infinity War,"Apr 27, 2018",Walt Disney,Action,"$678,815,482","74,513,225",https://www.the-numbers.com/movie/Avengers-Inf...,https://www.the-numbers.com/movie/Avengers-Inf...,2.63,300000000,6.8
2,3,Incredibles 2,"Jun 15, 2018",Walt Disney,Adventure,"$608,581,744","66,803,704",https://www.the-numbers.com/movie/Incredibles-...,https://www.the-numbers.com/movie/Incredibles-...,3.33,200000000,6.2
3,4,Jurassic World: Fallen Kingdom,"Jun 22, 2018",Universal,Action,"$417,719,760","45,852,883",https://www.the-numbers.com/movie/Jurassic-Wor...,https://www.the-numbers.com/movie/Jurassic-Wor...,2.82,170000000,7.7
4,5,Deadpool 2,"May 18, 2018",20th Century Fox,Action,"$324,512,774","35,621,600",https://www.the-numbers.com/movie/Deadpool-2#t...,https://www.the-numbers.com/movie/Deadpool-2#t...,2.59,110000000,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...
731,732,To Die is Hard,"Dec 17, 2010",Driving With Our …,Comedy,$580,63,https://www.the-numbers.com/movie/To-Die-is-Ha...,https://www.the-numbers.com/movie/To-Die-is-Ha...,3.45,nf,nf
732,733,Higher Power,"May 11, 2018",Magnet Pictures,Thriller/Suspense,$528,57,https://www.the-numbers.com/movie/Higher-Power...,https://www.the-numbers.com/movie/Higher-Power...,1.77,500000,0.0
733,734,Mobile Homes,"Apr 20, 2018",Mongrel Media,Drama,$449,49,https://www.the-numbers.com/movie/Mobile-Homes...,https://www.the-numbers.com/movie/Mobile-Homes...,1.00,nf,nf
734,735,Midget Zombie Takeover,"Feb 8, 2013",Driving With Our …,Comedy,$402,44,https://www.the-numbers.com/movie/Midget-Zombi...,https://www.the-numbers.com/movie/Midget-Zombi...,3.27,nf,nf


In [8]:
theater_count_2018_df = pd.read_pickle('theater_count_2018_pk')
avg_run_2018_df = pd.read_pickle('avg_run_2018_pk')

movies_final_2018_df['Theater_Count'] = theater_count_2018_df['Theater_Count']
movies_final_2018_df['Average_Run'] = avg_run_2018_df['Average_Run']


In [9]:
source_2018_df = pd.read_pickle('source_2018_pk')
movies_final_2018_df['Source'] = source_2018_df['Source']
# movies_final_2018_df

In [10]:
creative_type_2018_df =pd.read_pickle('creative_type_2018_pk') 
movies_final_2018_df['Creative_Type'] = creative_type_2018_df['Creative_Type']
movies_final_2018_df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,Source,Creative_Type
0,1,Black Panther,"Feb 16, 2018",Walt Disney,Action,"$700,059,566","76,845,177",https://www.the-numbers.com/movie/Black-Panthe...,https://www.the-numbers.com/movie/Black-Panthe...,3.47,200000000,6.7,4020,9.9,Based on Comic/Graphic Novel,Super Hero
1,2,Avengers: Infinity War,"Apr 27, 2018",Walt Disney,Action,"$678,815,482","74,513,225",https://www.the-numbers.com/movie/Avengers-Inf...,https://www.the-numbers.com/movie/Avengers-Inf...,2.63,300000000,6.8,4474,7.7,Based on Comic/Graphic Novel,Super Hero
2,3,Incredibles 2,"Jun 15, 2018",Walt Disney,Adventure,"$608,581,744","66,803,704",https://www.the-numbers.com/movie/Incredibles-...,https://www.the-numbers.com/movie/Incredibles-...,3.33,200000000,6.2,4410,9.1,Original Screenplay,Kids Fiction
3,4,Jurassic World: Fallen Kingdom,"Jun 22, 2018",Universal,Action,"$417,719,760","45,852,883",https://www.the-numbers.com/movie/Jurassic-Wor...,https://www.the-numbers.com/movie/Jurassic-Wor...,2.82,170000000,7.7,4475,6.8,Based on Fiction Book/Short Story,Science Fiction
4,5,Deadpool 2,"May 18, 2018",20th Century Fox,Action,"$324,512,774","35,621,600",https://www.the-numbers.com/movie/Deadpool-2#t...,https://www.the-numbers.com/movie/Deadpool-2#t...,2.59,110000000,7.1,4349,7.1,Based on Comic/Graphic Novel,Super Hero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,732,To Die is Hard,"Dec 17, 2010",Driving With Our …,Comedy,$580,63,https://www.the-numbers.com/movie/To-Die-is-Ha...,https://www.the-numbers.com/movie/To-Die-is-Ha...,3.45,nf,nf,1,5.0,Original Screenplay,Contemporary Fiction
732,733,Higher Power,"May 11, 2018",Magnet Pictures,Thriller/Suspense,$528,57,https://www.the-numbers.com/movie/Higher-Power...,https://www.the-numbers.com/movie/Higher-Power...,1.77,500000,0.0,3,1.0,Original Screenplay,Science Fiction
733,734,Mobile Homes,"Apr 20, 2018",Mongrel Media,Drama,$449,49,https://www.the-numbers.com/movie/Mobile-Homes...,https://www.the-numbers.com/movie/Mobile-Homes...,1.00,nf,nf,1,1.0,Original Screenplay,Contemporary Fiction
734,735,Midget Zombie Takeover,"Feb 8, 2013",Driving With Our …,Comedy,$402,44,https://www.the-numbers.com/movie/Midget-Zombi...,https://www.the-numbers.com/movie/Midget-Zombi...,3.27,nf,nf,1,3.0,Original Screenplay,Science Fiction


In [11]:
movies_final_2018_df.drop(['summary_page_url', 'cast_page_url'],
                          axis = 1, inplace = True)
movies_final_2018_df.shape

(736, 14)

In [12]:
col_reorder = ['Rank', 'Movie','ReleaseDate', 'Revenues', 'Tickets_Sold',
              'Legs', 'Budget', 'WBO_to_Budget_Ratio', 'Theater_Count',
               'Average_Run', 'Genre', 'Source', 'Creative_Type', 'Distributor']

movies_final_2018_df = movies_final_2018_df[col_reorder]

In [13]:
movies_final_2018_df

,Rank,Movie,ReleaseDate,Revenues,Tickets_Sold,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,Genre,Source,Creative_Type,Distributor
0,1,Black Panther,"Feb 16, 2018","$700,059,566","76,845,177",3.47,200000000,6.7,4020,9.9,Action,Based on Comic/Graphic Novel,Super Hero,Walt Disney
1,2,Avengers: Infinity War,"Apr 27, 2018","$678,815,482","74,513,225",2.63,300000000,6.8,4474,7.7,Action,Based on Comic/Graphic Novel,Super Hero,Walt Disney
2,3,Incredibles 2,"Jun 15, 2018","$608,581,744","66,803,704",3.33,200000000,6.2,4410,9.1,Adventure,Original Screenplay,Kids Fiction,Walt Disney
3,4,Jurassic World: Fallen Kingdom,"Jun 22, 2018","$417,719,760","45,852,883",2.82,170000000,7.7,4475,6.8,Action,Based on Fiction Book/Short Story,Science Fiction,Universal
4,5,Deadpool 2,"May 18, 2018","$324,512,774","35,621,600",2.59,110000000,7.1,4349,7.1,Action,Based on Comic/Graphic Novel,Super Hero,20th Century Fox
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,732,To Die is Hard,"Dec 17, 2010",$580,63,3.45,nf,nf,1,5.0,Comedy,Original Screenplay,Contemporary Fiction,Driving With Our …
732,733,Higher Power,"May 11, 2018",$528,57,1.77,500000,0.0,3,1.0,Thriller/Suspense,Original Screenplay,Science Fiction,Magnet Pictures
733,734,Mobile Homes,"Apr 20, 2018",$449,49,1.00,nf,nf,1,1.0,Drama,Original Screenplay,Contemporary Fiction,Mongrel Media
734,735,Midget Zombie Takeover,"Feb 8, 2013",$402,44,3.27,nf,nf,1,3.0,Comedy,Original Screenplay,Science Fiction,Driving With Our …


In [14]:
cast_2018_df = pd.read_pickle('cast_2018_pk')
# cast_2018_df

In [15]:
movies_final_2018_df = movies_final_2018_df.join(cast_2018_df)

In [16]:
movies_final_2018_df.to_pickle('final_2018_pk')

In [17]:
nf_count_dict = {}

# name = 'Budget'
# movies_final_2018_df[movies_final_2018_df[name] == 'nf'][name].count()

for name in col_reorder:
    count_nf = movies_final_2018_df[movies_final_2018_df[name] == 'nf'][name].count()
    print(f'{name}   {count_nf}')


    
    

Rank   0
Movie   0
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   510
WBO_to_Budget_Ratio   510
Theater_Count   47
Average_Run   47
Genre   0
Source   22
Creative_Type   16
Distributor   0


In [18]:
movies_final_2018_df.dtypes
# movies_final_2018_df

Rank                    object
Movie                   object
ReleaseDate             object
Revenues                object
Tickets_Sold            object
Legs                   float64
Budget                  object
WBO_to_Budget_Ratio     object
Theater_Count           object
Average_Run             object
Genre                   object
Source                  object
Creative_Type           object
Distributor             object
All_Cast_Rating        float64
Samuel L. Jackson        int64
Robert Downey, Jr.       int64
Scarlett Johansson       int64
Tom Hanks                int64
Bradley Cooper           int64
Harrison Ford            int64
Chris Evans              int64
Tom Cruise               int64
Chris Hemsworth          int64
Zoe Saldana              int64
dtype: object

In [19]:

movies_final_2018_df.replace(r'nf', np.nan, regex = True, inplace = True )


col_list = ['Revenues', 'Tickets_Sold', 'Legs', 'Budget', 'WBO_to_Budget_Ratio',
           'Theater_Count', 'Average_Run']

for col_name in col_list:
    movies_final_2018_df[col_name] = (movies_final_2018_df[col_name]
                                    .replace('[\$,]', '', regex = True))
                                        
    movies_final_2018_df[col_name] = (pd.to_numeric(movies_final_2018_df[col_name], 
                                                errors = 'coerce'))
    


In [20]:
movies_final_2018_df.dtypes
# movies_final_2018_df

Rank                    object
Movie                   object
ReleaseDate             object
Revenues                 int64
Tickets_Sold             int64
Legs                   float64
Budget                 float64
WBO_to_Budget_Ratio    float64
Theater_Count          float64
Average_Run            float64
Genre                   object
Source                  object
Creative_Type           object
Distributor             object
All_Cast_Rating        float64
Samuel L. Jackson        int64
Robert Downey, Jr.       int64
Scarlett Johansson       int64
Tom Hanks                int64
Bradley Cooper           int64
Harrison Ford            int64
Chris Evans              int64
Tom Cruise               int64
Chris Hemsworth          int64
Zoe Saldana              int64
dtype: object

In [21]:
for name in col_reorder:
    count_nf = movies_final_2018_df[movies_final_2018_df[name] == 'nf'][name].count()
    print(f'{name}   {count_nf}')

Rank   0
Movie   0
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   0
WBO_to_Budget_Ratio   0
Theater_Count   0
Average_Run   0
Genre   0
Source   0
Creative_Type   0
Distributor   0


In [22]:
for col_name in col_reorder:
    count_nan = movies_final_2018_df[col_name].isna().sum()
    print(f'{col_name}   {count_nan}')



Rank   0
Movie   3
ReleaseDate   0
Revenues   0
Tickets_Sold   0
Legs   0
Budget   510
WBO_to_Budget_Ratio   510
Theater_Count   47
Average_Run   47
Genre   0
Source   22
Creative_Type   16
Distributor   0


In [23]:
movies_final_2018_df.to_pickle('final_2018_pk')